In [1]:
import numpy as np
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import re
import os
import pickle

In [2]:
def list_imgs(directory, ext='jpg|jpeg|bmp|png|ppm'):
    return [os.path.join(root, f) for root, _, files in os.walk(directory) for f in files if re.match(r'([\w]+\.(?:' + ext + '))', f.lower())]

In [3]:
def plot_history(history, 
                save_graph_img_path, 
                fig_size_width, 
                fig_size_height, 
                lim_font_size):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
   
    epochs = range(len(acc))

    # グラフ表示
    plt.figure(figsize=(fig_size_width, fig_size_height))
    plt.rcParams['font.family'] = 'Times New Roman'
    plt.rcParams['font.size'] = lim_font_size  # 全体のフォント
    #plt.subplot(121)

    # plot accuracy values
    plt.plot(epochs, acc, color = "blue", linestyle = "solid", label = 'train acc')
    plt.plot(epochs, val_acc, color = "green", linestyle = "solid", label= 'valid acc')
    #plt.title('Training and Validation acc')
    #plt.grid()
    #plt.legend()
 
    # plot loss values
    #plt.subplot(122)
    plt.plot(epochs, loss, color = "red", linestyle = "solid" ,label = 'train loss')
    plt.plot(epochs, val_loss, color = "orange", linestyle = "solid" , label= 'valid loss')
    #plt.title('Training and Validation loss')
    plt.legend()
    plt.grid()

    plt.savefig(save_graph_img_path)
    plt.close() # バッファ解放


In [43]:
def main(epochNum):
  # バッチサイズ
  batch_size = 5
  # 分類クラス
  num_classes = 11

  # エポック数
  epochs = epochNum

  # 過学習防止
  dropout_rate = 0.2

  # 入力画像パラメータ
  img_width = 32
  img_height = 32
  img_ch = 3

  SAVE_DATA_DIR_PATH = ''

  FIG_SIZE_WIDTH = 12
  FIG_SIZE_HEIGHT = 10
  FIG_FONT_SIZE = 25

  data_x = []
  data_y = []

  for i in range(11): 
    for filepath in list_imgs(SAVE_DATA_DIR_PATH + str(i)):
      img = img_to_array(load_img(filepath, target_size=(img_width, img_height, img_ch)))
      data_x.append(img)
      data_y.append(i)

  # Numpy配列に変換
  data_x = np.asarray(data_x)
  data_y = np.asarray(data_y)

  # 分割
  x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size = 0.2)

  x_train = x_train.astype('float32')
  x_test = x_test.astype('float32')
  x_train = x_train / 255.0
  x_test = x_test / 255.0

  # 正解ラベルをone hotエンコーディング
  y_train = to_categorical(y_train, num_classes)
  y_test = to_categorical(y_test, num_classes)

  model = Sequential()

  model.add(Conv2D(32,(3,3),
            padding='same',
            input_shape=x_train.shape[1:],
            activation='relu'))
  
  model.add(Conv2D(32,(3,3),
                   padding='same',
                   activation='relu'))
  
  model.add(MaxPooling2D(pool_size = (2,2)))

  model.add(Dropout(dropout_rate))

  model.add(Conv2D(64, (3,3),
                   padding='same',
                   activation='relu'))
  
  model.add(Conv2D(64,(3,3),
                   padding='same',
                   activation='relu'))
  
  model.add(MaxPooling2D(pool_size=(2,2)))

  model.add(Dropout(dropout_rate))

  model.add(Flatten())

  model.add(Dense(512, activation='relu'))

  model.add(Dropout(dropout_rate))

  model.add(Dense(num_classes, activation='softmax'))

  model.summary()

  model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])


  history = model.fit(x_train, y_train,
                      batch_size=batch_size,
                      verbose = 1,
                      validation_split=0.1)

  # 構築したモデルで学習（学習データ:trainのうち、10％を検証データ:validationとして使用）
  history = model.fit(x_train, 
                      y_train, 
                      batch_size=batch_size, 
                      epochs=epochs, 
                      verbose=1, 
                      validation_split=0.1)
  
  score = model.evaluate(x_test, y_test, verbose=0)

  print('Test loss:', score[0])
  print('Test accuracy:', score[1])

  # モデル構造の保存
  open(SAVE_DATA_DIR_PATH  + "model_7.json","w").write(model.to_json())  

  # 学習済みの重みを保存
  model.save_weights(SAVE_DATA_DIR_PATH + "weight_7.hdf5")

  # 学習履歴を保存
  with open(SAVE_DATA_DIR_PATH + "history_7.json", 'wb') as f:
      pickle.dump(history.history, f)

  

In [44]:
main(4)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 conv2d_17 (Conv2D)          (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 16, 16, 32)       0         
 2D)                                                             
                                                                 
 dropout_12 (Dropout)        (None, 16, 16, 32)        0         
                                                                 
 conv2d_18 (Conv2D)          (None, 16, 16, 64)        18496     
                                                                 
 conv2d_19 (Conv2D)          (None, 16, 16, 64)        36928     
                                                      

In [50]:
# -*- coding: utf-8 -*-
import numpy as np
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def test():
    # 入力画像のパラメータ
    img_width = 32 # 入力画像の幅
    img_height = 32 # 入力画像の高さ
    img_ch = 3 # 3ch画像（RGB）

    # 入力データ数
    num_data = 1

    # データの保存先(自分の環境に応じて適宜変更)
    SAVE_DATA_DIR_PATH = ''

    # ラベル
    labels =['15', '1', '2', '3', '4','5','6','7','8','9','10','11','12','13','14']

    # 保存したモデル構造の読み込み
    model = model_from_json(open(SAVE_DATA_DIR_PATH + "model_7.json", 'r').read())

    # 保存した学習済みの重みを読み込み
    model.load_weights(SAVE_DATA_DIR_PATH + "weight_7.hdf5")

    # 画像の読み込み（32×32にリサイズ）
    # 正規化, 4次元配列に変換（モデルの入力が4次元なので合わせる）
    img = load_img(SAVE_DATA_DIR_PATH + "test.jpg", target_size=(img_width, img_height))
    img = img_to_array(img) 
    img = img.astype('float32')/255.0
    img = np.array([img])

    # 分類機に入力データを与えて予測（出力：各クラスの予想確率）
    y_pred = model.predict(img)

    # 最も確率の高い要素番号
    number_pred = np.argmax(y_pred) 

    # 予測結果の表示
    print("y_pred:", y_pred)  # 出力値
    # if 0.8 < y_pred[0][number_pred]:
    print("number_pred:", number_pred)  # 最も確率の高い要素番号
    print('label_pred：', labels[int(number_pred)]) # 予想ラベル（最も確率の高い要素）
    # else:
    #     print("number_pred:99")  # 最も確率の高い要素番号
    #     print('label_pred：99') # 予想ラベル（最も確率の高い要素）

In [51]:
test()

1/1 [==============================] - 0s 62ms/step
y_pred: [[8.0015504e-08 1.3105891e-03 9.0196114e-03 1.1074079e-03 1.0075931e-04
  8.7927980e-03 8.0096489e-03 9.0953995e-07 5.7045484e-01 4.0108511e-01
  1.1822014e-04]]
number_pred: 8
label_pred： 8
